In [47]:
import pandas as pd
import pyodbc
from datamart import fix_me

In [5]:
username = 'vigrose'
password_aims = 'DQA8K@T7pXEqEt'
s = "DSN=aims_prod; UID={}; PWD={}".format(username, password_aims)
informix = pyodbc.connect(s)

In [14]:
password_edw = 'Gryffindor~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)

In [8]:
query = \
    f'''
    SELECT
    S.SRC_CONF_CAT_CODE,
    S.SRC_CONF_CATEGORY,
    U.ENTITY_ID,
    U.COMM_ID,
    U.SRC_CAT_CODE,
    U.SRC_CATEGORY,
    U.UPDATE_DTM
    FROM
    informix.SRC_CONF_DTL_GT S, informix.ENTITY_COMM_USG_AT U
    WHERE
    S.SRC_CONF_CAT_CODE = 'PHONE-CALL'
    AND
    S.APPROVAL_ID = U.APPROVAL_ID
    AND
    U.END_DT IS NULL
    AND 
    U.SRC_CONF_BY 
    '''

In [9]:
usc_approvals = pd.read_sql(con=informix, sql=query)
usc_approvals.head()

,src_conf_cat_code,src_conf_category,entity_id,comm_id,src_cat_code,src_category,update_dtm
0,PHONE-CALL,SRC-ADDR,10488773,6733112,PHONE-CALL,SRC-ADDR,2012-01-24 12:47:46.764
1,PHONE-CALL,SRC-ADDR,13698408,6732616,PHONE-CALL,SRC-ADDR,2005-04-07 02:26:00.599
2,PHONE-CALL,SRC-ADDR,3552116,6198351,PHONE-CALL,SRC-ADDR,2009-05-14 10:53:34.163
3,PHONE-CALL,SRC-ADDR,12839485,6166747,PHONE-CALL,SRC-ADDR,2005-07-27 23:54:19.443
4,PHONE-CALL,SRC-ADDR,12786510,9820233,PHONE-CALL,SRC-ADDR,2012-11-09 14:20:44.748


In [10]:
len(usc_approvals)

359562

In [12]:
usc_approvals['YEAR']=[x.year for x in usc_approvals.update_dtm]

In [13]:
usc_approvals.groupby('YEAR').count()

,src_conf_cat_code,src_conf_category,entity_id,comm_id,src_cat_code,src_category,update_dtm
YEAR,,,,,,,
1999,506,506,506,506,506,506,506
2000,2917,2917,2917,2917,2917,2917,2917
2001,3010,3010,3010,3010,3010,3010,3010
2002,4919,4919,4919,4919,4919,4919,4919
2003,8924,8924,8924,8924,8923,8923,8924
2004,13508,13508,13508,13508,13491,13491,13508
2005,11223,11223,11223,11223,11207,11207,11223
2006,17707,17707,17707,17707,17674,17674,17707
2007,10497,10497,10497,10497,10483,10483,10497


In [17]:
ADDRESSES = \
    f'''
    SELECT DISTINCT
    A.PARTY_ID,
    A.POST_CD_ID,
    P.ADDR_1,
    P.ADDR_2,
    P.ADDR_3,
    P.CITY,
    S.SRC_STATE_CD AS STATE,
    P.POST_CD AS ZIP,
    A.FROM_DT,
    P.SRC_POST_KEY AS POST_KEY
    FROM
    AMAEDW.POST_CD P, AMAEDW.STATE S , AMAEDW.PARTY_ADDR A, AMAEDW.CONT_PURPOSE_TYPE C, AMAEDW.CAT_CD CC
    WHERE
    A.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
    AND
    C.PURPOSE_CAT_CD='A'
    AND
    C.PURPOSE_USG_CD='PO'
    AND
    A.POST_CD_ID = P.POST_CD_ID
    AND
    P.STATE_ID = S.STATE_ID
    AND
    A.THRU_DT IS NULL
    AND
    CC.CAT_CD_ID = A.CAT_CD_ID;
    '''
addresses = pd.read_sql(con=AMAEDW, sql=ADDRESSES)
addresses.head()

ENTITY_ID = \
    """
    SELECT DISTINCT
    P.PARTY_ID,
    P.KEY_VAL AS ENTITY_ID
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = '9'
    """
entity_ids = pd.read_sql(con=AMAEDW, sql=ENTITY_ID)
entity_ids.head()

#Get party ids
ME_QUERY = \
    """
    SELECT DISTINCT
    P.PARTY_ID,
    P.KEY_VAL AS ME
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    AND
    P.ACTIVE_IND = 'Y'
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

,PARTY_ID,POST_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE,ZIP,FROM_DT,POST_KEY
0,2135294,15279367,7701 State Line Rd,,,Kansas City,MO,64114,2016-10-18 07:22:59.042096,1286324
1,2664009,10292981,1175 Montauk Hwy Ste 1,,,West Islip,NY,11795,2016-10-18 07:22:59.445103,3288931
2,2336726,13691337,405 Northfield Ave,Ste 204,,West Orange,NJ,07052,2016-10-18 07:22:59.950862,5445470
3,2537472,14738419,Medical Center Blvd,Wake Forest Univ Dept Rad,,Winston Salem,NC,27157,2016-10-18 07:23:00.628196,6153569
4,2717698,12341036,7300 SW 62nd Pl,Fl 2,,South Miami,FL,33143,2016-10-18 07:23:00.662517,6162980


In [18]:
entity_ids = \
    """
    SELECT DISTINCT
    P.PARTY_ID,
    P.KEY_VAL AS ENTITY_ID
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = '9'
    """
ENTITY_ID = pd.read_sql(con=AMAEDW, sql=entity_ids)
ENTITY_ID.head()

#Get party ids
ME_QUERY = \
    """
    SELECT DISTINCT
    P.PARTY_ID,
    P.KEY_VAL AS ME
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    AND
    P.ACTIVE_IND = 'Y'
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

,PARTY_ID,ENTITY_ID
0,7556000,46715886
1,7556001,46715899
2,7556002,46715912
3,7556003,46715925
4,7556004,46715938


In [38]:
#Get party ids
ME_QUERY = \
    """
    SELECT DISTINCT
    P.PARTY_ID,
    P.KEY_VAL AS ME
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    AND
    P.ACTIVE_IND = 'Y'
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [32]:
usc_approvals['ENTITY_ID'] = usc_approvals.ENTITY_ID.astype(str)

In [34]:
usc_approvals.columns = [c.upper() for c in usc_approvals.columns.values]
usc = pd.merge(usc_approvals, ENTITY_ID, on='ENTITY_ID')

In [39]:
usc_addresses = pd.merge(usc, addresses, left_on=['PARTY_ID','COMM_ID'], right_on = ['PARTY_ID','POST_KEY']).drop_duplicates()

In [42]:
usc_addresses.groupby('YEAR').count()

,SRC_CONF_CAT_CODE,SRC_CONF_CATEGORY,ENTITY_ID,COMM_ID,SRC_CAT_CODE,SRC_CATEGORY,UPDATE_DTM,PARTY_ID,POST_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE,ZIP,FROM_DT,POST_KEY
YEAR,,,,,,,,,,,,,,,,,
1999,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2001,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
2002,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8
2003,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11
2004,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23
2005,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21,21
2006,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31
2007,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15
2008,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33,33


In [43]:
usc_addresses = pd.merge(usc_addresses, party_ids, on='PARTY_ID')

In [44]:
humach = pd.read_csv('../../Data/Humach/humach_updated_since_2018.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [48]:
humach['ME']=fix_me(humach.PHYSICIAN_ME_NUMBER)

In [54]:
pd.merge(usc_addresses, humach, on='ME')[['ADDR_1','OFFICE_ADDRESS_LINE_2','FROM_DT','SOURCE_FILE_DT']]

,ADDR_1,OFFICE_ADDRESS_LINE_2,FROM_DT,SOURCE_FILE_DT
0,30 N 1900 E # 4A330,30 N 1900 E # 4A330,2018-03-23 21:03:56.322947,03/27/2020
1,30 N 1900 E # 4A330,30 N 1900 E # 4A330,2018-03-23 21:03:56.322947,NaN
2,2508 Western Ave,2508 WEST AVE,2018-10-12 21:00:39.431496,NaN
3,2508 Western Ave,2508 WESTERN AVE,2018-10-12 21:00:39.431496,04/28/2020
4,1430 Tulane Ave # 8036,1430 TULANE AVE # 8036,2017-09-27 20:44:06.699679,03/27/2020
...,...,...,...,...
5231,1000 N Oak Ave,1000 N OAK AVE STE 2D,2017-02-07 21:59:22.459816,07/27/2020
5232,1000 N Oak Ave,1000 N OAK AVE STE 2D,2017-02-07 21:59:22.459816,07/27/2020
5233,11921 Saradrienne Ln,11921 SARADRIENNE LN,2017-09-01 20:46:57.897838,NaN
5234,735 S D St,735 S D ST,2019-03-24 20:41:34.568815,NaN


In [46]:
humach.columns

Index(['PHYSICIAN_ME_NUMBER', 'PHYSICIAN_FIRST_NAME', 'PHYSICIAN_MIDDLE_NAME',
       'PHYSICIAN_LAST_NAME', 'SUFFIX', 'DEGREE', 'OFFICE_ADDRESS_LINE_1',
       'OFFICE_ADDRESS_LINE_2', 'OFFICE_ADDRESS_CITY', 'OFFICE_ADDRESS_STATE',
       'OFFICE_ADDRESS_ZIP', 'OFFICE_ADDRESS_VERIFIED_UPDATED',
       'OFFICE_TELEPHONE', 'OFFICE_PHONE_VERIFIED_UPDATED', 'OFFICE_FAX',
       'OFFICE_FAX_VERIFIED_UPDATED', 'EMAIL_ADDRESS',
       'EMAIL_ADDRESS_VERIFIED_UPDATED', 'PRESENT_EMPLOYMENT_CODE',
       'PRESENT_EMPLOYMENT_UPDATED', 'COMMENTS', 'Source', 'WSLIVE_SOURCE',
       'WSLIVE_FILE_DT', 'MATCH_ADDR', 'MATCH_PHONE', 'MATCH_ADDR_LONG',
       'SPECIALTY', 'SPECIALTY_UPDATED', 'SOURCE_FILE_DT'],
      dtype='object')

In [50]:
usc_addresses[usc_addresses.YEAR==2021]

,SRC_CONF_CAT_CODE,SRC_CONF_CATEGORY,ENTITY_ID,COMM_ID,SRC_CAT_CODE,SRC_CATEGORY,UPDATE_DTM,YEAR,PARTY_ID,POST_CD_ID,ADDR_1,ADDR_2,ADDR_3,CITY,STATE,ZIP,FROM_DT,POST_KEY,ME
64,PHONE-CALL,SRC-ADDR,9921336,20474169,PHONE-CALL,SRC-ADDR,2021-04-22 22:49:09.506,2021,2573868,26347656,1801 Parkview Dr,Indian Health Clinic,,El Reno,OK,73036,2021-04-23 07:16:14.428344,20474169,04204920180
68,PHONE-CALL,SRC-ADDR,13462224,25655038,PHONE-CALL,SRC-ADDR,2021-02-03 15:00:05.882,2021,2816595,32025498,2557 S 800 E,,,Salt Lake Cty,UT,84106,2021-02-03 21:10:25.846711,25655038,60501840176
74,PHONE-CALL,SRC-ADDR,3462923,9880740,PHNSURV,SRC,2021-01-10 13:22:25.508,2021,2016885,12119656,130 S Main St Ste 206,,,Lombard,IL,60148,2021-01-10 21:10:51.799031,9880740,01611893899
123,PHONE-CALL,SRC-ADDR,1879211,25628216,PHONE-CALL,SRC-ADDR,2021-01-20 12:01:33.196,2021,1858608,31998336,789 Medical Center Dr East,,,Clovis,CA,93611,2021-01-20 21:10:10.895694,25628216,00514871299
124,PHONE-CALL,SRC-ADDR,1879211,25628216,PHONE-CALL,SRC-ADDR,2021-01-20 12:01:33.306,2021,1858608,31998336,789 Medical Center Dr East,,,Clovis,CA,93611,2021-01-20 21:10:10.895694,25628216,00514871299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14893,PHONE-CALL,SRC-ADDR,47148032,25067418,None,None,2021-04-22 23:57:03.100,2021,7804375,31467391,130 Division St,,,Derby,CT,06418,2021-04-23 07:16:28.349952,25067418,51705120033
14894,PHONE-CALL,SRC-ADDR,47148032,25067418,PHONE-CALL,SRC-ADDR,2021-04-22 23:57:03.100,2021,7804375,31467391,130 Division St,,,Derby,CT,06418,2021-04-23 07:16:28.349952,25067418,51705120033
14896,PHONE-CALL,SRC-ADDR,21548900,25823863,PHONE-CALL,SRC-ADDR,2021-04-29 23:48:14.679,2021,3360621,32209323,1115 West Paul Street,Florida State University College of Med,,Tallahassee,FL,32306,2021-04-30 07:17:57.361400,25823863,30501010495
14898,PHONE-CALL,SRC-ADDR,14959070,804598,PHNSURV,SRC,2021-01-10 13:22:27.393,2021,2856530,9689958,3330 Lomita Blvd,,,Torrance,CA,90505,2021-01-10 21:10:38.372234,804598,00515020799


In [52]:
humach.columns

Index(['PHYSICIAN_ME_NUMBER', 'PHYSICIAN_FIRST_NAME', 'PHYSICIAN_MIDDLE_NAME',
       'PHYSICIAN_LAST_NAME', 'SUFFIX', 'DEGREE', 'OFFICE_ADDRESS_LINE_1',
       'OFFICE_ADDRESS_LINE_2', 'OFFICE_ADDRESS_CITY', 'OFFICE_ADDRESS_STATE',
       'OFFICE_ADDRESS_ZIP', 'OFFICE_ADDRESS_VERIFIED_UPDATED',
       'OFFICE_TELEPHONE', 'OFFICE_PHONE_VERIFIED_UPDATED', 'OFFICE_FAX',
       'OFFICE_FAX_VERIFIED_UPDATED', 'EMAIL_ADDRESS',
       'EMAIL_ADDRESS_VERIFIED_UPDATED', 'PRESENT_EMPLOYMENT_CODE',
       'PRESENT_EMPLOYMENT_UPDATED', 'COMMENTS', 'Source', 'WSLIVE_SOURCE',
       'WSLIVE_FILE_DT', 'MATCH_ADDR', 'MATCH_PHONE', 'MATCH_ADDR_LONG',
       'SPECIALTY', 'SPECIALTY_UPDATED', 'SOURCE_FILE_DT', 'ME'],
      dtype='object')

In [68]:
humach_fails = pd.read_csv('../../Data/Humach/humach_fails_since_2018.csv', encoding='latin')
humach_fails.head()

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4,10,16,25,27,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,PHYSICIAN_ME_NUMBER,PHYSICIAN_FIRST_NAME,PHYSICIAN_MIDDLE_NAME,PHYSICIAN_LAST_NAME,SUFFIX,DEGREE,OFFICE_ADDRESS_LINE_1,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,...,COMMENTS,Source,WSLIVE_SOURCE,WSLIVE_FILE_DT,MATCH_ADDR,MATCH_PHONE,MATCH_ADDR_LONG,SPECIALTY,SPECIALTY_UPDATED,SOURCE_FILE_DT
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,FAIL,NaN,FAIL,04/30/2020,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,FAIL,NaN,FAIL,04/10/2020,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,FAIL,NaN,FAIL,03/31/2020,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,FAIL,NaN,FAIL,03/31/2020,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,FAIL,NaN,FAIL,01/31/2020,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
humach_fails['ME']=fix_me(humach_fails.PHYSICIAN_ME_NUMBER)

In [71]:
test = pd.merge(usc_addresses, humach_fails, on= 'ME')

In [75]:
test['DATE'] = pd.to_datetime(test.WSLIVE_FILE_DT)

In [78]:
this_bitch = test.sort_values('DATE').drop_duplicates('ME', keep='last')

In [80]:
yeses = []
for row in this_bitch.itertuples():
    if row.DATE < row.UPDATE_DTM:
        YES = True
    else:
        YES = False
    yeses.append(YES)

In [81]:
this_bitch['NEWER']=yeses

In [86]:
this_bitch[this_bitch.NEWER==True][['ADDR_1','OFFICE_ADDRESS_LINE_2','YEAR', 'WSLIVE_FILE_DT','COMMENTS']]

,ADDR_1,OFFICE_ADDRESS_LINE_2,YEAR,WSLIVE_FILE_DT,COMMENTS
4585,1122 N Topeka St,4722 W KELLOGG DR,2019,05/31/2018,FAIL
4565,230 E Ridgewood Ave,800 POLY PL,2019,05/31/2018,FAIL
3044,1200 Memorial Dr,1200 MEMORIAL DR,2019,05/31/2018,FAIL
2909,7900 Airways Blvd Ste 2,7900 AIRWAYS BLVD,2020,05/31/2018,FAIL
716,400 S Oyster Bay Rd Ste 307,120 BETHPAGE RD,2019,05/31/2018,FAIL
...,...,...,...,...,...
429,24075 Commerce Park,2013 STATE ROUTE 59,2021,02/28/2021,FAIL
6891,26840 Point Lookout Rd,26840 POINT LOOKOUT RD,2021,03/31/2021,FAIL
6592,419 Vernon St,912 PARK AVE,2021,03/31/2021,FAIL
5284,200 1st St SW,200 1ST ST SW,2021,03/31/2021,FAIL


In [87]:
humach_other = pd.read_csv('../../Data/Humach/humach_bad_since_2018.csv', encoding='latin')
humach_other.head()

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (10,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,PHYSICIAN_ME_NUMBER,PHYSICIAN_FIRST_NAME,PHYSICIAN_MIDDLE_NAME,PHYSICIAN_LAST_NAME,SUFFIX,DEGREE,OFFICE_ADDRESS_LINE_1,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,...,COMMENTS,Source,WSLIVE_SOURCE,WSLIVE_FILE_DT,MATCH_ADDR,MATCH_PHONE,MATCH_ADDR_LONG,SPECIALTY,SPECIALTY_UPDATED,SOURCE_FILE_DT
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,OTHERS,06/30/2018,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,OTHERS,07/13/2018,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,OTHERS,09/14/2018,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,OTHERS,07/27/2018,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,OTHERS,08/24/2018,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
humach_other['ME']=fix_me(humach_other.PHYSICIAN_ME_NUMBER)
test_2 = pd.merge(usc_addresses, humach_other, on= 'ME')
test_2['DATE'] = pd.to_datetime(test_2.WSLIVE_FILE_DT)
this_bitch_2 = test_2.sort_values('DATE').drop_duplicates('ME', keep='last')

In [89]:
this_bitch_2

,SRC_CONF_CAT_CODE,SRC_CONF_CATEGORY,ENTITY_ID,COMM_ID,SRC_CAT_CODE,SRC_CATEGORY,UPDATE_DTM,YEAR,PARTY_ID,POST_CD_ID,...,Source,WSLIVE_SOURCE,WSLIVE_FILE_DT,MATCH_ADDR,MATCH_PHONE,MATCH_ADDR_LONG,SPECIALTY,SPECIALTY_UPDATED,SOURCE_FILE_DT,DATE
2516,PHONE-CALL,SRC-ADDR,3454798,22542345,PHONE-CALL,SRC-ADDR,2018-08-15 09:03:01.329,2018,1946273,28987380,...,Q,OTHERS,05/25/2018,0161187425971860628,1611874257732334180,NaN,MPD,1.0,NaN,2018-05-25
2589,PHONE-CALL,SRC-ADDR,43000343,11830077,DEA,SRC-ADDR,2017-10-05 23:14:19.774,2017,5601642,19561384,...,S,OTHERS,05/25/2018,0260413168190056303,2604131683202294924,NaN,D,1.0,NaN,2018-05-25
235,PHONE-CALL,SRC-ADDR,16082088,20901302,DEA,SRC-ADDR,2016-11-16 23:06:59.726,2016,2940111,26899284,...,S,OTHERS,05/25/2018,0480403036499978249,04804030362105614433,NaN,GS,1.0,NaN,2018-05-25
356,PHONE-CALL,SRC-ADDR,13164706,21943562,DEA,SRC-ADDR,2017-09-28 22:19:42.579,2017,2654548,28100558,...,C,OTHERS,05/25/2018,5171086001541560640,51710860013126249783,NaN,APM,1.0,NaN,2018-05-25
1424,PHONE-CALL,SRC-ADDR,7443445,20998270,OTHER,SRC-ADDR,2017-03-24 13:03:51.557,2017,2198403,26997256,...,Q,OTHERS,05/25/2018,0350999119390190814,3509991195624348421,NaN,PM,1.0,NaN,2018-05-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,PHONE-CALL,SRC-ADDR,10871961,21167077,DEA,SRC-ADDR,2018-10-18 22:37:06.678,2018,2597361,27180090,...,QM,OTHERS,04/30/2021,0481683014240176508,04816830142547242111,NaN,OTO,1.0,04/05/2021,2021-04-30
1009,PHONE-CALL,SRC-ADDR,6627357,21587672,PHONE-CALL,SRC-ADDR,2017-05-24 10:35:19.342,2017,2325931,27780428,...,Q,OTHERS,04/30/2021,0300681085595050266,03006810855158759100,NaN,PS,1.0,04/05/2021,2021-04-30
1844,PHONE-CALL,SRC-ADDR,2845826,18618376,DEA,SRC-ADDR,2016-12-08 19:32:03.108,2016,1879110,23167280,...,QM,OTHERS,04/30/2021,0120190094170730474,01201900949125373384,NaN,OTO,1.0,04/05/2021,2021-04-30
1530,PHONE-CALL,SRC-ADDR,4043958,23687791,DEA,SRC-ADDR,2019-07-25 22:32:11.114,2019,2103874,30000239,...,Q,OTHERS,04/30/2021,018038402260056267,01803840223205896070,NaN,FM,1.0,04/05/2021,2021-04-30


In [90]:
yeses = []
for row in this_bitch_2.itertuples():
    if row.DATE < row.UPDATE_DTM:
        YES = True
    else:
        YES = False
    yeses.append(YES)
this_bitch_2['NEWER']=yeses

In [98]:
this_bitch_2[this_bitch_2.NEWER==True][['SRC_CONF_CATEGORY','ADDR_1','OFFICE_ADDRESS_LINE_2','UPDATE_DTM', 'WSLIVE_FILE_DT','COMMENTS']]

,SRC_CONF_CATEGORY,ADDR_1,OFFICE_ADDRESS_LINE_2,UPDATE_DTM,WSLIVE_FILE_DT,COMMENTS
2516,SRC-ADDR,9817 S Halsted St,9718 S HALSTED ST,2018-08-15 09:03:01.329,05/25/2018,"MOVED, NO FORWARDING INFO"
641,SRC-ADDR,715 Richland Mall,1 PERKINS SQ,2019-07-21 13:24:17.559,05/25/2018,REFUSAL
1413,SRC-ADDR,550 S Caldwell St Ste 1500,9 WAPPING RD,2018-10-25 22:09:51.378,05/25/2018,FAX MODEM
2493,SRC-ADDR,5916 East Lake Pkwy,241 RALPH MCGILL BLVD NE,2019-05-08 09:17:40.903,06/01/2018,REFUSAL
2346,SRC-ADDR,303 S Main St,1100 REID PKWY,2018-08-09 10:18:23.599,06/01/2018,"MOVED, NO FORWARDING INFO"
...,...,...,...,...,...,...
983,SRC-ADDR,1705 Warren Ave,1705 WARREN AVE,2021-04-22 22:43:52.875,02/12/2021,"MOVED, NO FORWARDING INFO"
1605,SRC-ADDR,7727 Lake Underhill Rd,10179 RIDGEBLOOM AVE,2021-03-25 15:05:16.825,02/19/2021,WRONG NUMBER
1122,SRC-ADDR,854 Washington Ave,1919 BOSTON ST SE,2021-04-16 08:04:37.497,03/12/2021,"MOVED, NO FORWARDING INFO"
1909,SRC-ADDR,1510 San Pablo St Ste 514,NaN,2021-04-22 22:47:08.562,04/01/2021,NOT IN SERVICE


In [95]:
this_bitch.columns

Index(['SRC_CONF_CAT_CODE', 'SRC_CONF_CATEGORY', 'ENTITY_ID', 'COMM_ID',
       'SRC_CAT_CODE', 'SRC_CATEGORY', 'UPDATE_DTM', 'YEAR', 'PARTY_ID',
       'POST_CD_ID', 'ADDR_1', 'ADDR_2', 'ADDR_3', 'CITY', 'STATE', 'ZIP',
       'FROM_DT', 'POST_KEY', 'ME', 'PHYSICIAN_ME_NUMBER',
       'PHYSICIAN_FIRST_NAME', 'PHYSICIAN_MIDDLE_NAME', 'PHYSICIAN_LAST_NAME',
       'SUFFIX', 'DEGREE', 'OFFICE_ADDRESS_LINE_1', 'OFFICE_ADDRESS_LINE_2',
       'OFFICE_ADDRESS_CITY', 'OFFICE_ADDRESS_STATE', 'OFFICE_ADDRESS_ZIP',
       'OFFICE_ADDRESS_VERIFIED_UPDATED', 'OFFICE_TELEPHONE',
       'OFFICE_PHONE_VERIFIED_UPDATED', 'OFFICE_FAX',
       'OFFICE_FAX_VERIFIED_UPDATED', 'EMAIL_ADDRESS',
       'EMAIL_ADDRESS_VERIFIED_UPDATED', 'PRESENT_EMPLOYMENT_CODE',
       'PRESENT_EMPLOYMENT_UPDATED', 'COMMENTS', 'Source', 'WSLIVE_SOURCE',
       'WSLIVE_FILE_DT', 'MATCH_ADDR', 'MATCH_PHONE', 'MATCH_ADDR_LONG',
       'SPECIALTY', 'SPECIALTY_UPDATED', 'SOURCE_FILE_DT', 'DATE', 'NEWER'],
      dtype='object')